# Naive Bayes Classifier cho bài toán spam filtering (phân loại thư rác)

Mô tả dữ liệu Tập dữ liệu này gồm 960 email tiếng Anh, được tách thành traning set và test set theo tỉ lệ 700:260 với 50% trong mỗi tập là các spam email.

Dữ liệu trong cơ sở dữ liệu này đã được xử lý. Các quy tắc xử lý:

1. Loại bỏ stop words: loại bỏ những từ không cần thiết, không ảnh hưởng nhiều đến việc quyết định
2. Lemmatization: đưa những từ gốc về cùng loại.
3. Loại bỏ non-words: loại bỏ các chữ số, kí tự, dấu câu.

Danh sách các file txt:

train-features-50.txt: chứa dữ liệu của tranning set thu gọn với 50 email.

labels.txt: chứa nhiều dòng, mỗi dòng là một kí tự 0 hoặc 1 thể hiện email non-spam hoặc spam.

features.txt: chứa nhiều dòng, mỗi dòng gồm 3 số:

Vd: 1 564 1, 1 19 2.

Trong đó số đầu tiên là chỉ số của email, số thứ hai là thứ tự của từ trong từ điển, số thứ ba là số lượng từ đó trong email đang xét. 1 564 1 , email 1, từ thứ 564, xuất hiện trong từ điển 1 lần. Nếu biểu diễn mỗi email bằng một vector hàng có độ dài từ điển 2500 thì dòng thứ nhất nói rằng đặc trưng thứ 564 của vector này bằng 1, tương tự đặc trưng thứ 19 của vector này bằng 2.
    

In [1]:
from __future__ import print_function
import numpy as np
from scipy.sparse import coo_matrix # for sparse matrix
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.metrics import accuracy_score # for evaluating results

# data path and file name 
path = 'ex6DataPrepared/'
train_data_fn = 'train-features.txt'
test_data_fn = 'test-features.txt'
train_label_fn = 'train-labels.txt'
test_label_fn = 'test-labels.txt'

Xem dữ liệu


Label của dữ liệu

In [33]:
with open(path + train_label_fn) as f:
    content = f.readlines()
    label = [int(x.strip()) for x in content]
print(label)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

features của dữ liệu

In [44]:
with open(path + train_data_fn) as f:
    content = f.readlines()
    #print("Dữ liệu đọc được:",content)
    # remove '\n' at the end of each line
    content = [x.strip() for x in content] 
    #print("\nDữ liệu:\n")
    #print(content)
    

Đưa dữ liệu features về chuẩn

In [49]:
dat = np.zeros((len(content), 3), dtype = int)
for i, line in enumerate(content): 
        a = line.split(' ')
        dat[i, :] = np.array([int(a[0]), int(a[1]), int(a[2])])
print(dat)

[[   1   19    2]
 [   1   45    1]
 [   1   50    1]
 ...
 [ 700 2479    2]
 [ 700 2481    2]
 [ 700 2500    3]]


Đưa dữ liệu về coo_matrix

In [50]:
data = coo_matrix((dat[:, 2], (dat[:, 0] - 1, dat[:, 1] - 1)),shape=(len(label), nwords))
print(data)

  (0, 18)	2
  (0, 44)	1
  (0, 49)	1
  (0, 74)	1
  (0, 84)	1
  (0, 138)	1
  (0, 199)	1
  (0, 350)	1
  (0, 351)	1
  (0, 512)	1
  (0, 563)	1
  (0, 742)	1
  (0, 776)	1
  (0, 1130)	1
  (0, 1276)	1
  (0, 1638)	1
  (0, 1763)	1
  (0, 1815)	1
  (0, 1867)	1
  (1, 34)	5
  (1, 96)	1
  (1, 102)	2
  (1, 158)	1
  (1, 293)	2
  (1, 726)	1
  :	:
  (699, 2048)	4
  (699, 2054)	4
  (699, 2057)	2
  (699, 2071)	4
  (699, 2084)	2
  (699, 2108)	3
  (699, 2126)	1
  (699, 2172)	1
  (699, 2198)	3
  (699, 2226)	1
  (699, 2231)	1
  (699, 2236)	1
  (699, 2244)	1
  (699, 2325)	1
  (699, 2356)	3
  (699, 2377)	2
  (699, 2397)	4
  (699, 2401)	1
  (699, 2418)	1
  (699, 2432)	1
  (699, 2433)	1
  (699, 2471)	1
  (699, 2478)	2
  (699, 2480)	2
  (699, 2499)	3


In [22]:
nwords = 2500 

def read_data(data_fn, label_fn):
    ## read label_fn
    with open(path + label_fn) as f:
        content = f.readlines()
    label = [int(x.strip()) for x in content]

    ## read data_fn
    with open(path + data_fn) as f:
        content = f.readlines()
    # remove '\n' at the end of each line
    content = [x.strip() for x in content] 

    dat = np.zeros((len(content), 3), dtype = int)
    
    for i, line in enumerate(content): 
        a = line.split(' ')
        dat[i, :] = np.array([int(a[0]), int(a[1]), int(a[2])])
    
    # remember to -1 at coordinate since we're in Python
    # check this: https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.coo_matrix.html
    # for more information about coo_matrix function 
    data = coo_matrix((dat[:, 2], (dat[:, 0] - 1, dat[:, 1] - 1)),\
             shape=(len(label), nwords))
    return (data, label)

In [25]:
(train_data, train_label)  = read_data(train_data_fn, train_label_fn)
(test_data, test_label)  = read_data(test_data_fn, test_label_fn)
print(train_data)
print(train_label)
clf = MultinomialNB()
clf.fit(train_data, train_label)

y_pred = clf.predict(test_data)
print('Training size = %d, accuracy = %.2f%%' % \
      (train_data.shape[0],accuracy_score(test_label, y_pred)*100))

  (0, 18)	2
  (0, 44)	1
  (0, 49)	1
  (0, 74)	1
  (0, 84)	1
  (0, 138)	1
  (0, 199)	1
  (0, 350)	1
  (0, 351)	1
  (0, 512)	1
  (0, 563)	1
  (0, 742)	1
  (0, 776)	1
  (0, 1130)	1
  (0, 1276)	1
  (0, 1638)	1
  (0, 1763)	1
  (0, 1815)	1
  (0, 1867)	1
  (1, 34)	5
  (1, 96)	1
  (1, 102)	2
  (1, 158)	1
  (1, 293)	2
  (1, 726)	1
  :	:
  (699, 2048)	4
  (699, 2054)	4
  (699, 2057)	2
  (699, 2071)	4
  (699, 2084)	2
  (699, 2108)	3
  (699, 2126)	1
  (699, 2172)	1
  (699, 2198)	3
  (699, 2226)	1
  (699, 2231)	1
  (699, 2236)	1
  (699, 2244)	1
  (699, 2325)	1
  (699, 2356)	3
  (699, 2377)	2
  (699, 2397)	4
  (699, 2401)	1
  (699, 2418)	1
  (699, 2432)	1
  (699, 2433)	1
  (699, 2471)	1
  (699, 2478)	2
  (699, 2480)	2
  (699, 2499)	3
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

Sử dụng mô hình Multinomial navie Bayes cho dữ liệu training nhỏ hơn 100 email

In [51]:
train_data_fn = 'train-features-100.txt'
train_label_fn = 'train-labels-100.txt'
test_data_fn = 'test-features.txt'
test_label_fn = 'test-labels.txt'

(train_data, train_label)  = read_data(train_data_fn, train_label_fn)
(test_data, test_label)  = read_data(test_data_fn, test_label_fn)
clf = MultinomialNB()
clf.fit(train_data, train_label)
y_pred = clf.predict(test_data)
print('Training size = %d, accuracy = %.2f%%' % \
      (train_data.shape[0],accuracy_score(test_label, y_pred)*100))

train_data_fn = 'train-features-50.txt'
train_label_fn = 'train-labels-50.txt'
test_data_fn = 'test-features.txt'
test_label_fn = 'test-labels.txt'

(train_data, train_label)  = read_data(train_data_fn, train_label_fn)
(test_data, test_label)  = read_data(test_data_fn, test_label_fn)
clf = MultinomialNB()
clf.fit(train_data, train_label)
y_pred = clf.predict(test_data)
print('Training size = %d, accuracy = %.2f%%' % \
      (train_data.shape[0],accuracy_score(test_label, y_pred)*100))

Training size = 100, accuracy = 97.69%
Training size = 50, accuracy = 97.31%


Sử dụng mô hình Bernoulli

In [52]:
clf = BernoulliNB(binarize = .5)
clf.fit(train_data, train_label)
y_pred = clf.predict(test_data)
print('Training size = %d, accuracy = %.2f%%' % \
      (train_data.shape[0],accuracy_score(test_label, y_pred)*100))

Training size = 50, accuracy = 69.62%
